In [2]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input, Bidirectional, Dense, Conv1D, Conv3D, LSTM, ConvLSTM1D, ConvLSTM2D, Flatten, Concatenate, TimeDistributed
from utilz import *
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

2022-10-12 11:34:26.218976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 11:34:26.305417: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-12 11:34:26.325928: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 11:34:26.715101: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [3]:
visual_ori = load_features('./data/visual_mpori.pkl')
visual_of = load_features('./data/visual_OFfts.pkl')
visual_clip = load_features('./data/visual_clip.pkl')
label = load_features('./data/labels.pkl')

In [22]:
print(np.shape(visual_ori['train']),
      np.shape(visual_of['train']),
      np.shape(visual_clip['train']))

(1368, 10, 64, 64) (1368, 30, 49) (1368, 10, 512)


# Model 1

In [4]:
x = Input((10, 64, 64, 1))
h = Conv3D(16, [3,3,3], [2,2,2], 'same')(x)
h = Conv3D(32, [1,1,1], [2,2,2], 'same')(h)
h = Conv3D(8, [3,3,3], [2,2,2], 'same')(h)

h = Flatten()(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 64, 64, 1)]   0         
                                                                 
 conv3d (Conv3D)             (None, 5, 32, 32, 16)     448       
                                                                 
 conv3d_1 (Conv3D)           (None, 3, 16, 16, 32)     544       
                                                                 
 conv3d_2 (Conv3D)           (None, 2, 8, 8, 8)        6920      
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 3)                 3075      
                                                                 
Total params: 10,987
Trainable params: 10,987
Non-trainable p

2022-10-06 12:19:45.335570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 12:19:45.363957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 12:19:45.364100: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 12:19:45.364439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [5]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNN_mpori.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.expand_dims(visual_ori['train'], axis=-1), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.expand_dims(visual_ori['valid'], axis=-1), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30


2022-10-06 12:19:50.158569: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


 1/86 [..............................] - ETA: 2:10 - loss: 29.3258 - acc: 0.1875

2022-10-06 12:19:50.585265: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


71/86 [=======================>......] - ETA: 0s - loss: 8.6513 - acc: 0.4085

...

INFO:tensorflow:Assets written to: ./res/V_model_CNN_mpori.tf/assets


86/86 [==============================] - 1s 7ms/step - loss: 0.9960 - acc: 0.6908 - val_loss: 2.7931 - val_acc: 0.4189
Epoch 5/30
71/86 [=======================>......] - ETA: 0s - loss: 0.6678 - acc: 0.7623

INFO:tensorflow:Assets written to: ./res/V_model_CNN_mpori.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNN_mpori.tf/assets


86/86 [==============================] - 1s 8ms/step - loss: 0.6754 - acc: 0.7610 - val_loss: 2.6713 - val_acc: 0.4320
Epoch 6/30
86/86 [==============================] - 0s 3ms/step - loss: 0.4920 - acc: 0.8077 - val_loss: 2.8357 - val_acc: 0.4649
Epoch 7/30
86/86 [==============================] - 0s 3ms/step - loss: 0.3632 - acc: 0.8662 - val_loss: 2.7742 - val_acc: 0.4276
Epoch 8/30
86/86 [==============================] - 0s 3ms/step - loss: 0.2591 - acc: 0.9057 - val_loss: 2.7736 - val_acc: 0.4320
Epoch 9/30
86/86 [==============================] - 0s 3ms/step - loss: 0.2105 - acc: 0.9247 - val_loss: 2.8930 - val_acc: 0.4496
Epoch 10/30
86/86 [==============================] - 0s 3ms/step - loss: 0.1716 - acc: 0.9459 - val_loss: 2.9733 - val_acc: 0.4408
Epoch 11/30
86/86 [==============================] - 0s 3ms/step - loss: 0.1184 - acc: 0.9722 - val_loss: 3.2057 - val_acc: 0.4232
Epoch 12/30
86/86 [==============================] - 0s 3ms/step - loss: 0.0917 - acc: 0.9817 - val

In [10]:
model = tf.keras.models.load_model('./res/V_model_CNN_mpori.tf/')
pred = model.predict(np.expand_dims(visual_ori['test'], axis=-1))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         Neg     0.5294    0.4355    0.4779       248
         Pos     0.3046    0.3286    0.3162       140
         Neu     0.1275    0.1884    0.1520        69

    accuracy                         0.3654       457
   macro avg     0.3205    0.3175    0.3154       457
weighted avg     0.3999    0.3654    0.3791       457

[[108  82  58]
 [ 63  46  31]
 [ 33  23  13]]


In [60]:
x = Input((30, 49))
h = Conv1D(64, 3, 1, 'same')(x)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = Flatten()(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 30, 49)]          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 64)            9472      
                                                                 
 conv1d_2 (Conv1D)           (None, 30, 64)            4160      
                                                                 
 conv1d_3 (Conv1D)           (None, 30, 64)            12352     
                                                                 
 flatten_22 (Flatten)        (None, 1920)              0         
                                                                 
 dense_55 (Dense)            (None, 3)                 5763      
                                                                 
Total params: 31,747
Trainable params: 31,747
Non-trainabl

In [61]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNN_of.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_of['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_of['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
86/86 [==============================] - ETA: 0s - loss: 32.5679 - acc: 0.4145

INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


86/86 [==============================] - 1s 10ms/step - loss: 32.5679 - acc: 0.4145 - val_loss: 4.5351 - val_acc: 0.3289
Epoch 2/30
86/86 [==============================] - 0s 2ms/step - loss: 5.0057 - acc: 0.4437 - val_loss: 9.1261 - val_acc: 0.1667
Epoch 3/30
73/86 [========================>.....] - ETA: 0s - loss: 4.5452 - acc: 0.4264

INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


...

INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


86/86 [==============================] - 1s 6ms/step - loss: 1.5575 - acc: 0.5673 - val_loss: 1.6044 - val_acc: 0.5219
Epoch 15/30
64/86 [=====================>........] - ETA: 0s - loss: 0.9546 - acc: 0.6357

INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNN_of.tf/assets


86/86 [==============================] - 1s 6ms/step - loss: 0.9721 - acc: 0.6287 - val_loss: 1.3928 - val_acc: 0.5504
Epoch 16/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9603 - acc: 0.6396 - val_loss: 1.6377 - val_acc: 0.4890
Epoch 17/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9065 - acc: 0.6411 - val_loss: 1.8931 - val_acc: 0.5461
Epoch 18/30
86/86 [==============================] - 0s 2ms/step - loss: 1.0143 - acc: 0.6279 - val_loss: 1.4633 - val_acc: 0.5022
Epoch 19/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9988 - acc: 0.6557 - val_loss: 2.0323 - val_acc: 0.3684
Epoch 20/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9491 - acc: 0.6411 - val_loss: 2.5594 - val_acc: 0.5329
Epoch 21/30
86/86 [==============================] - 0s 2ms/step - loss: 0.9588 - acc: 0.6396 - val_loss: 1.8058 - val_acc: 0.4956
Epoch 22/30
86/86 [==============================] - 0s 2ms/step - loss: 1.0515 - acc: 0.6265 -

In [63]:
model = tf.keras.models.load_model('./res/V_model_CNN_of.tf/')
pred = model.predict(np.asarray(visual_of['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

         Neg     0.5800    0.8185    0.6789       248
         Pos     0.5647    0.3429    0.4267       140
         Neu     0.0455    0.0145    0.0220        69

    accuracy                         0.5514       457
   macro avg     0.3967    0.3920    0.3759       457
weighted avg     0.4946    0.5514    0.5025       457

[[203  32  13]
 [ 84  48   8]
 [ 63   5   1]]


In [64]:
x = Input((10, 512))
h = Conv1D(64, 3, 1, 'same')(x)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = Flatten()(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 10, 512)]         0         
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 64)            98368     
                                                                 
 conv1d_5 (Conv1D)           (None, 10, 64)            4160      
                                                                 
 conv1d_6 (Conv1D)           (None, 10, 64)            12352     
                                                                 
 flatten_23 (Flatten)        (None, 640)               0         
                                                                 
 dense_56 (Dense)            (None, 3)                 1923      
                                                                 
Total params: 116,803
Trainable params: 116,803
Non-traina

In [65]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNN_clip.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_clip['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_clip['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
86/86 [==============================] - ETA: 0s - loss: 1.0114 - acc: 0.5322

INFO:tensorflow:Assets written to: ./res/V_model_CNN_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNN_clip.tf/assets


86/86 [==============================] - 1s 10ms/step - loss: 1.0114 - acc: 0.5322 - val_loss: 0.9768 - val_acc: 0.5921
Epoch 2/30
71/86 [=======================>......] - ETA: 0s - loss: 0.7901 - acc: 0.6549

INFO:tensorflow:Assets written to: ./res/V_model_CNN_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNN_clip.tf/assets


86/86 [==============================] - 1s 7ms/step - loss: 0.7865 - acc: 0.6550 - val_loss: 0.9093 - val_acc: 0.5899
Epoch 3/30
86/86 [==============================] - 0s 4ms/step - loss: 0.6385 - acc: 0.7325 - val_loss: 0.9950 - val_acc: 0.5921
Epoch 4/30
86/86 [==============================] - 0s 5ms/step - loss: 0.4983 - acc: 0.8034 - val_loss: 1.1365 - val_acc: 0.5351
Epoch 5/30
86/86 [==============================] - 0s 5ms/step - loss: 0.3582 - acc: 0.8713 - val_loss: 1.3861 - val_acc: 0.5285
Epoch 6/30
86/86 [==============================] - 0s 4ms/step - loss: 0.2568 - acc: 0.9167 - val_loss: 1.5751 - val_acc: 0.5351
Epoch 7/30
86/86 [==============================] - 0s 2ms/step - loss: 0.1673 - acc: 0.9393 - val_loss: 1.8574 - val_acc: 0.5219
Epoch 8/30
86/86 [==============================] - 0s 2ms/step - loss: 0.1352 - acc: 0.9539 - val_loss: 2.3118 - val_acc: 0.5088
Epoch 9/30
86/86 [==============================] - 0s 2ms/step - loss: 0.0803 - acc: 0.9788 - val_lo

In [66]:
model = tf.keras.models.load_model('./res/V_model_CNN_clip.tf/')
pred = model.predict(np.asarray(visual_clip['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         Neg     0.6700    0.8185    0.7368       248
         Pos     0.5571    0.5571    0.5571       140
         Neu     0.1429    0.0290    0.0482        69

    accuracy                         0.6193       457
   macro avg     0.4567    0.4682    0.4474       457
weighted avg     0.5558    0.6193    0.5778       457

[[203  39   6]
 [ 56  78   6]
 [ 44  23   2]]


# Model 2

In [23]:
x = Input((10, 64, 64, 1))
h = ConvLSTM2D(16, [3,3], [2,2], 'same', return_sequences=True)(x)
h = TimeDistributed(Flatten())(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 10, 64, 64, 1)]   0         
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 10, 32, 32, 16)    9856      
                                                                 
 time_distributed_4 (TimeDis  (None, 10, 16384)        0         
 tributed)                                                       
                                                                 
 lstm_2 (LSTM)               (None, 64)                4210944   
                                                                 
 dense_10 (Dense)            (None, 3)                 195       
                                                                 
Total params: 4,220,995
Trainable params: 4,220,995
Non-trainable params: 0
________________________________________________

In [24]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTM_mpori.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.expand_dims(visual_ori['train'], axis=-1), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.expand_dims(visual_ori['valid'], axis=-1), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
86/86 [==============================] - ETA: 0s - loss: 1.7863 - acc: 0.4825INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_mpori.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_mpori.tf/assets


86/86 [==============================] - 7s 65ms/step - loss: 1.7863 - acc: 0.4825 - val_loss: 1.0580 - val_acc: 0.5044
Epoch 2/30
85/86 [============================>.] - ETA: 0s - loss: 0.7934 - acc: 0.6551INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_mpori.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_mpori.tf/assets


86/86 [==============================] - 5s 56ms/step - loss: 0.7935 - acc: 0.6542 - val_loss: 1.0413 - val_acc: 0.5044
Epoch 3/30
86/86 [==============================] - 3s 34ms/step - loss: 0.5683 - acc: 0.7763 - val_loss: 1.2113 - val_acc: 0.5219
Epoch 4/30
86/86 [==============================] - 3s 33ms/step - loss: 0.3272 - acc: 0.8984 - val_loss: 1.2213 - val_acc: 0.4759
Epoch 5/30
86/86 [==============================] - 3s 37ms/step - loss: 0.2196 - acc: 0.9430 - val_loss: 1.5425 - val_acc: 0.4978
Epoch 6/30
86/86 [==============================] - 3s 38ms/step - loss: 0.1923 - acc: 0.9539 - val_loss: 1.6878 - val_acc: 0.5351
Epoch 7/30
86/86 [==============================] - 3s 40ms/step - loss: 0.1182 - acc: 0.9744 - val_loss: 1.7958 - val_acc: 0.4232
Epoch 8/30
86/86 [==============================] - 3s 39ms/step - loss: 0.0586 - acc: 0.9905 - val_loss: 1.9997 - val_acc: 0.4803
Epoch 9/30
86/86 [==============================] - 3s 34ms/step - loss: 0.0264 - acc: 0.9920 

In [25]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTM_mpori.tf/')
pred = model.predict(np.expand_dims(visual_ori['test'], axis=-1))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 10ms/step
              precision    recall  f1-score   support

         Neg     0.5681    0.7903    0.6610       248
         Pos     0.3553    0.1929    0.2500       140
         Neu     0.2500    0.1304    0.1714        69

    accuracy                         0.5077       457
   macro avg     0.3911    0.3712    0.3608       457
weighted avg     0.4549    0.5077    0.4612       457

[[196  33  19]
 [105  27   8]
 [ 44  16   9]]


In [92]:
x = Input((30, 49))
h = Conv1D(64, 3, 1, 'same')(x)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_35 (InputLayer)       [(None, 30, 49)]          0         
                                                                 
 conv1d_30 (Conv1D)          (None, 30, 64)            9472      
                                                                 
 conv1d_31 (Conv1D)          (None, 30, 64)            4160      
                                                                 
 conv1d_32 (Conv1D)          (None, 30, 64)            12352     
                                                                 
 lstm_20 (LSTM)              (None, 64)                33024     
                                                                 
 dense_65 (Dense)            (None, 3)                 195       
                                                                 
Total params: 59,203
Trainable params: 59,203
Non-trainabl

In [93]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTM_of.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_of['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_of['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
85/86 [============================>.] - ETA: 0s - loss: 1429.7815 - acc: 0.3603

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


86/86 [==============================] - 5s 50ms/step - loss: 1424.3860 - acc: 0.3604 - val_loss: 710.1916 - val_acc: 0.3947
Epoch 2/30
86/86 [==============================] - ETA: 0s - loss: 557.7405 - acc: 0.4050

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


...

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


86/86 [==============================] - 3s 29ms/step - loss: 17.1485 - acc: 0.4547 - val_loss: 13.7250 - val_acc: 0.3904
Epoch 25/30
86/86 [==============================] - 2s 19ms/step - loss: 19.9645 - acc: 0.4466 - val_loss: 18.5867 - val_acc: 0.5175
Epoch 26/30
86/86 [==============================] - 2s 20ms/step - loss: 16.2776 - acc: 0.4335 - val_loss: 15.6038 - val_acc: 0.4890
Epoch 27/30
86/86 [==============================] - 2s 20ms/step - loss: 17.4524 - acc: 0.4444 - val_loss: 17.1900 - val_acc: 0.4978
Epoch 28/30
86/86 [==============================] - 2s 19ms/step - loss: 13.7274 - acc: 0.4349 - val_loss: 15.2358 - val_acc: 0.5066
Epoch 29/30
86/86 [==============================] - 2s 20ms/step - loss: 12.5611 - acc: 0.4444 - val_loss: 15.6363 - val_acc: 0.3925
Epoch 30/30
84/86 [============================>.] - ETA: 0s - loss: 17.4361 - acc: 0.4159

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_of.tf/assets


86/86 [==============================] - 3s 29ms/step - loss: 17.2630 - acc: 0.4181 - val_loss: 11.8334 - val_acc: 0.4759


In [94]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTM_of.tf/')
pred = model.predict(np.asarray(visual_of['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

         Neg     0.5670    0.8185    0.6700       248
         Pos     0.3816    0.2071    0.2685       140
         Neu     0.0435    0.0145    0.0217        69

    accuracy                         0.5098       457
   macro avg     0.3307    0.3467    0.3201       457
weighted avg     0.4312    0.5098    0.4491       457

[[203  31  14]
 [103  29   8]
 [ 52  16   1]]


In [95]:
x = Input((10, 512))
h = Conv1D(64, 3, 1, 'same')(x)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_36 (InputLayer)       [(None, 10, 512)]         0         
                                                                 
 conv1d_33 (Conv1D)          (None, 10, 64)            98368     
                                                                 
 conv1d_34 (Conv1D)          (None, 10, 64)            4160      
                                                                 
 conv1d_35 (Conv1D)          (None, 10, 64)            12352     
                                                                 
 lstm_21 (LSTM)              (None, 64)                33024     
                                                                 
 dense_66 (Dense)            (None, 3)                 195       
                                                                 
Total params: 148,099
Trainable params: 148,099
Non-traina

In [96]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTM_clip.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_clip['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_clip['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
81/86 [===========================>..] - ETA: 0s - loss: 0.9608 - acc: 0.5509

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


86/86 [==============================] - 2s 22ms/step - loss: 0.9507 - acc: 0.5548 - val_loss: 0.9078 - val_acc: 0.5877
Epoch 2/30
86/86 [==============================] - 1s 11ms/step - loss: 0.8625 - acc: 0.6323 - val_loss: 0.9237 - val_acc: 0.5921
Epoch 3/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8213 - acc: 0.6637 - val_loss: 0.9559 - val_acc: 0.5636
Epoch 4/30
86/86 [==============================] - 1s 14ms/step - loss: 0.7717 - acc: 0.6762 - val_loss: 1.0107 - val_acc: 0.6206
Epoch 5/30
85/86 [============================>.] - ETA: 0s - loss: 0.7520 - acc: 0.6794

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


86/86 [==============================] - 2s 22ms/step - loss: 0.7521 - acc: 0.6798 - val_loss: 0.8990 - val_acc: 0.6184
Epoch 6/30
86/86 [==============================] - ETA: 0s - loss: 0.7100 - acc: 0.6820

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTM_clip.tf/assets


86/86 [==============================] - 2s 23ms/step - loss: 0.7100 - acc: 0.6820 - val_loss: 0.8728 - val_acc: 0.6250
Epoch 7/30
86/86 [==============================] - 1s 10ms/step - loss: 0.6748 - acc: 0.7127 - val_loss: 0.9418 - val_acc: 0.6031
Epoch 8/30
86/86 [==============================] - 1s 13ms/step - loss: 0.6331 - acc: 0.7222 - val_loss: 0.9458 - val_acc: 0.6096
Epoch 9/30
86/86 [==============================] - 1s 11ms/step - loss: 0.6093 - acc: 0.7332 - val_loss: 0.9933 - val_acc: 0.5855
Epoch 10/30
86/86 [==============================] - 1s 10ms/step - loss: 0.5692 - acc: 0.7500 - val_loss: 1.1048 - val_acc: 0.5987
Epoch 11/30
86/86 [==============================] - 1s 10ms/step - loss: 0.5635 - acc: 0.7463 - val_loss: 1.5224 - val_acc: 0.6228
Epoch 12/30
86/86 [==============================] - 1s 10ms/step - loss: 0.5820 - acc: 0.7485 - val_loss: 1.0548 - val_acc: 0.6075
Epoch 13/30
86/86 [==============================] - 1s 10ms/step - loss: 0.5111 - acc: 0.7

In [98]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
pred = model.predict(np.asarray(visual_clip['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

         Neg     0.6210    0.9315    0.7452       248
         Pos     0.6849    0.3571    0.4695       140
         Neu     0.5000    0.0870    0.1481        69

    accuracy                         0.6280       457
   macro avg     0.6020    0.4585    0.4543       457
weighted avg     0.6223    0.6280    0.5706       457

[[231  13   4]
 [ 88  50   2]
 [ 53  10   6]]


# Model 3

In [5]:
class Attention_Self(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(Attention_Self, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.S = tf.keras.layers.Dense(1)
        self.units = units

    def call(self, features):
        features_ = tf.expand_dims(features, 1)
        v = self.W1(features)
        q =  self.W2(features_)
        score = tf.nn.tanh(q + v)
        attention_weights = tf.nn.softmax(self.S(score), axis=1)
        # ATTN = attention_weights * (v)
        ATTN = attention_weights * (v)
        ATTN = tf.reduce_sum(ATTN, axis=1)
        
        return ATTN
        
    def get_config(self):
        config = super(Attention_Self, self).get_config()
        config.update({"units": self.units})
        return config

In [52]:
x = Input((10, 64, 64, 1))
h = ConvLSTM2D(16, [3,3], [2,2], 'same', return_sequences=True)(x)
h = Attention_Self(16)(h)
h = TimeDistributed(Flatten())(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 10, 64, 64, 1)]   0         
                                                                 
 conv_lstm2d_19 (ConvLSTM2D)  (None, 10, 32, 32, 16)   9856      
                                                                 
 attention__self_11 (Attenti  (None, 10, 32, 32, 16)   561       
 on_Self)                                                        
                                                                 
 time_distributed_16 (TimeDi  (None, 10, 16384)        0         
 stributed)                                                      
                                                                 
 lstm_10 (LSTM)              (None, 64)                4210944   
                                                                 
 dense_54 (Dense)            (None, 3)                 195

In [53]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTMAtt_mpori.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.expand_dims(visual_ori['train'], axis=-1), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.expand_dims(visual_ori['valid'], axis=-1), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
85/86 [============================>.] - ETA: 0s - loss: 1.8604 - acc: 0.4926

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


86/86 [==============================] - 8s 78ms/step - loss: 1.8545 - acc: 0.4942 - val_loss: 1.0729 - val_acc: 0.5044
Epoch 2/30
86/86 [==============================] - ETA: 0s - loss: 1.0093 - acc: 0.5358

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


...

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


86/86 [==============================] - 6s 68ms/step - loss: 0.9876 - acc: 0.5424 - val_loss: 0.9830 - val_acc: 0.5439
Epoch 13/30
86/86 [==============================] - ETA: 0s - loss: 0.9812 - acc: 0.5424

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_mpori.tf/assets


86/86 [==============================] - 6s 75ms/step - loss: 0.9812 - acc: 0.5424 - val_loss: 0.9814 - val_acc: 0.5439
Epoch 14/30
86/86 [==============================] - 5s 54ms/step - loss: 1.0139 - acc: 0.5395 - val_loss: 0.9882 - val_acc: 0.5439
Epoch 15/30
86/86 [==============================] - 4s 52ms/step - loss: 0.9956 - acc: 0.5395 - val_loss: 0.9894 - val_acc: 0.5439
Epoch 16/30
86/86 [==============================] - 4s 45ms/step - loss: 0.9980 - acc: 0.5424 - val_loss: 0.9873 - val_acc: 0.5439
Epoch 17/30
86/86 [==============================] - 4s 45ms/step - loss: 0.9879 - acc: 0.5424 - val_loss: 1.0246 - val_acc: 0.5395
Epoch 18/30
86/86 [==============================] - 4s 45ms/step - loss: 0.9899 - acc: 0.5424 - val_loss: 1.0287 - val_acc: 0.5439
Epoch 19/30
86/86 [==============================] - 4s 45ms/step - loss: 0.9819 - acc: 0.5424 - val_loss: 0.9910 - val_acc: 0.5439
Epoch 20/30
86/86 [==============================] - 4s 47ms/step - loss: 0.9834 - acc: 

In [55]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTMAtt_mpori.tf/')
pred = model.predict(np.expand_dims(visual_ori['test'], axis=-1))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 27ms/step
              precision    recall  f1-score   support

         Neg     0.5427    1.0000    0.7035       248
         Pos     0.0000    0.0000    0.0000       140
         Neu     0.0000    0.0000    0.0000        69

    accuracy                         0.5427       457
   macro avg     0.1809    0.3333    0.2345       457
weighted avg     0.2945    0.5427    0.3818       457

[[248   0   0]
 [140   0   0]
 [ 69   0   0]]


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [17]:
x = Input((30, 49))
h = Attention_Self(64)(x)
h = Conv1D(64, 3, 1, 'same')(h)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 30, 49)]          0         
                                                                 
 attention__self_4 (Attentio  (None, 30, 64)           6465      
 n_Self)                                                         
                                                                 
 conv1d_12 (Conv1D)          (None, 30, 64)            12352     
                                                                 
 conv1d_13 (Conv1D)          (None, 30, 64)            4160      
                                                                 
 conv1d_14 (Conv1D)          (None, 30, 64)            12352     
                                                                 
 lstm_6 (LSTM)               (None, 64)                33024     
                                                           

In [18]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTMAtt_of.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_of['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_of['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
84/86 [============================>.] - ETA: 0s - loss: 56.4657 - acc: 0.3460

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


86/86 [==============================] - 4s 38ms/step - loss: 55.5201 - acc: 0.3465 - val_loss: 1.8686 - val_acc: 0.4890
Epoch 2/30
83/86 [===========================>..] - ETA: 0s - loss: 1.3809 - acc: 0.4239

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


...

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


86/86 [==============================] - 3s 39ms/step - loss: 1.0674 - acc: 0.4817 - val_loss: 1.2194 - val_acc: 0.5000
Epoch 25/30
86/86 [==============================] - 2s 23ms/step - loss: 1.1042 - acc: 0.4846 - val_loss: 1.2267 - val_acc: 0.5088
Epoch 26/30
86/86 [==============================] - 2s 22ms/step - loss: 1.0974 - acc: 0.4788 - val_loss: 1.2357 - val_acc: 0.4561
Epoch 27/30
86/86 [==============================] - 2s 21ms/step - loss: 1.1058 - acc: 0.5022 - val_loss: 1.2784 - val_acc: 0.3355
Epoch 28/30
86/86 [==============================] - 2s 22ms/step - loss: 1.1123 - acc: 0.4700 - val_loss: 1.2350 - val_acc: 0.5329
Epoch 29/30
85/86 [============================>.] - ETA: 0s - loss: 1.1091 - acc: 0.4838

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_of.tf/assets


86/86 [==============================] - 3s 35ms/step - loss: 1.1103 - acc: 0.4832 - val_loss: 1.1901 - val_acc: 0.5044
Epoch 30/30
86/86 [==============================] - 2s 21ms/step - loss: 1.1421 - acc: 0.4846 - val_loss: 1.1971 - val_acc: 0.5329


In [19]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTMAtt_of.tf/')
pred = model.predict(np.asarray(visual_of['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 6ms/step
              precision    recall  f1-score   support

         Neg     0.5351    0.9516    0.6851       248
         Pos     0.1333    0.0143    0.0258       140
         Neu     0.0000    0.0000    0.0000        69

    accuracy                         0.5208       457
   macro avg     0.2228    0.3220    0.2370       457
weighted avg     0.3313    0.5208    0.3797       457

[[236  12   0]
 [137   2   1]
 [ 68   1   0]]


In [20]:
x = Input((10, 512))
h = Attention_Self(512)(x)
h = Conv1D(64, 3, 1, 'same')(h)
h = Conv1D(64, 1, 1, 'same')(h)
h = Conv1D(64, 3, 1, 'same')(h)
h = LSTM(64, activation='relu')(h)
res = Dense(3, 'softmax')(h)

model = Model(inputs=x, outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 10, 512)]         0         
                                                                 
 attention__self_5 (Attentio  (None, 10, 512)          525825    
 n_Self)                                                         
                                                                 
 conv1d_15 (Conv1D)          (None, 10, 64)            98368     
                                                                 
 conv1d_16 (Conv1D)          (None, 10, 64)            4160      
                                                                 
 conv1d_17 (Conv1D)          (None, 10, 64)            12352     
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                           

In [21]:
callback_list = [ModelCheckpoint(filepath='./res/V_model_CNNLSTMAtt_clip.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=np.asarray(visual_clip['train']), y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[np.asarray(visual_clip['valid']), np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
82/86 [===========================>..] - ETA: 0s - loss: 1.0669 - acc: 0.5061

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_clip.tf/assets


86/86 [==============================] - 4s 34ms/step - loss: 1.0632 - acc: 0.5073 - val_loss: 0.9842 - val_acc: 0.5439
Epoch 2/30
86/86 [==============================] - 1s 17ms/step - loss: 0.9838 - acc: 0.5461 - val_loss: 0.9999 - val_acc: 0.5461
Epoch 3/30
86/86 [==============================] - 1s 13ms/step - loss: 0.9669 - acc: 0.5490 - val_loss: 0.9955 - val_acc: 0.5570
Epoch 4/30
86/86 [==============================] - 1s 12ms/step - loss: 0.9248 - acc: 0.5870 - val_loss: 1.0004 - val_acc: 0.5241
Epoch 5/30
86/86 [==============================] - 1s 12ms/step - loss: 0.8859 - acc: 0.6148 - val_loss: 1.0277 - val_acc: 0.5373
Epoch 6/30
86/86 [==============================] - 1s 13ms/step - loss: 0.8749 - acc: 0.6250 - val_loss: 1.0532 - val_acc: 0.5680
Epoch 7/30
85/86 [============================>.] - ETA: 0s - loss: 0.9104 - acc: 0.6221

INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_clip.tf/assets


INFO:tensorflow:Assets written to: ./res/V_model_CNNLSTMAtt_clip.tf/assets


86/86 [==============================] - 2s 26ms/step - loss: 0.9113 - acc: 0.6221 - val_loss: 0.9741 - val_acc: 0.5504
Epoch 8/30
86/86 [==============================] - 1s 12ms/step - loss: 1.0167 - acc: 0.5000 - val_loss: 0.9939 - val_acc: 0.5570
Epoch 9/30
86/86 [==============================] - 1s 12ms/step - loss: 0.9650 - acc: 0.5680 - val_loss: 1.6236 - val_acc: 0.5439
Epoch 10/30
86/86 [==============================] - 1s 12ms/step - loss: 0.9108 - acc: 0.5980 - val_loss: 1.0181 - val_acc: 0.5504
Epoch 11/30
86/86 [==============================] - 1s 12ms/step - loss: 0.8157 - acc: 0.6696 - val_loss: 1.2824 - val_acc: 0.4978
Epoch 12/30
86/86 [==============================] - 1s 16ms/step - loss: 0.9338 - acc: 0.5702 - val_loss: 1.0673 - val_acc: 0.5439
Epoch 13/30
86/86 [==============================] - 2s 18ms/step - loss: 1.0534 - acc: 0.5424 - val_loss: 1.0393 - val_acc: 0.5439
Epoch 14/30
86/86 [==============================] - 1s 16ms/step - loss: 1.0299 - acc: 0.

In [22]:
model = tf.keras.models.load_model('./res/V_model_CNNLSTMAtt_clip.tf/')
pred = model.predict(np.asarray(visual_clip['test']))
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

         Neg     0.5427    1.0000    0.7035       248
         Pos     0.0000    0.0000    0.0000       140
         Neu     0.0000    0.0000    0.0000        69

    accuracy                         0.5427       457
   macro avg     0.1809    0.3333    0.2345       457
weighted avg     0.2945    0.5427    0.3818       457

[[248   0   0]
 [140   0   0]
 [ 69   0   0]]


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le